In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os
import sys
# sys.path.insert(0, '/Users/austin/Documents/GitHub/pyTEMlib')

# sys.path.insert(0, '/Users/austin/Documents/GitHub/SciFiReaders')
# import SciFiReaders
import pyTEMlib
from pyTEMlib import file_tools as ft
from pyTEMlib import image_tools as it
%matplotlib ipympl


from scipy.spatial import KDTree
import matplotlib.cm as cm

In [ ]:
print(pyTEMlib.__version__)

## Graphene example

In [ ]:
# Graphene exp data
path = '/Users/austin/Dropbox/graphene_images/Hao_Wang_colab/'
files = os.listdir(path)
files = [f for f in files if f.endswith('.emd')]

In [ ]:
dates = [file.split(' ')[2].split('_')[0] for file in files]
dates = list(set(dates))

date = dates[0]
sorted_files = [file for file in files if '.emd' in file]

# file indexes 2,8,14 on dates[4] are Moires

In [ ]:
file = sorted_files[14]

dset = ft.open_file(path + file)
image = dset['Channel_000']
viw = image.plot(cmap='gray')

pixel_size = image.original_metadata['BinaryResult']['PixelSize']['width']
pixel_size = np.array(pixel_size)
print('Pixel size: ', pixel_size, )

In [ ]:
im_array = np.array(image)

np.savez('Moire.npz', im_array=im_array, pixel_size=pixel_size)

In [ ]:
power_spectrum = pyTEMlib.image_tools.power_spectrum(image, smoothing=1)

power_spectrum.view_metadata()
view = power_spectrum.plot()

In [ ]:
# find spots in power spectrum
# import blob finder
from skimage.feature import blob_log


In [ ]:
# find blobs
blobs = blob_log(power_spectrum, min_sigma=1, max_sigma=10, num_sigma=10, threshold=0.5)

# plot blobs

fig, ax = plt.subplots()
ax.imshow(power_spectrum, cmap='gray')
for blob in blobs:
    y, x, r = blob
    c = plt.Circle((x, y), r, color='red', linewidth=2, fill=False)
    ax.add_patch(c)

ax.set_xlim(power_spectrum.shape[1]//3, 2*power_spectrum.shape[1]//3)
ax.set_ylim(power_spectrum.shape[0]//3, 2*power_spectrum.shape[0]//3)
ax.axis('off')

In [ ]:
# get inner blobs:
inner_blobs = blob_log(power_spectrum, min_sigma=1, max_sigma=10, num_sigma=10, threshold=0.52)
# filter by distance from center
# center is blob closest to COM of the blobs
center = np.array([power_spectrum.shape[0]//2, power_spectrum.shape[1]//2])
distances = [np.linalg.norm(np.array([blob[0], blob[1]]) - center) for blob in inner_blobs]
inner_blobs = [inner_blobs[i] for i in range(len(inner_blobs)) if distances[i] < 30]
inner_blobs = [inner_blobs[i] for i in range(len(inner_blobs)) if distances[i] > 18]

# get outer blobs
outer_blobs = blob_log(power_spectrum, min_sigma=1, max_sigma=10, num_sigma=10, threshold=0.5)
# filter by distance from center
# center is blob closest to COM of the blobs
center = np.array([power_spectrum.shape[0]//2, power_spectrum.shape[1]//2])
distances = [np.linalg.norm(np.array([blob[0], blob[1]]) - center) for blob in outer_blobs]
outer_blobs = [outer_blobs[i] for i in range(len(outer_blobs)) if distances[i] > 30]


In [ ]:
all_blobs = inner_blobs + outer_blobs

In [ ]:
plt.figure()
plt.imshow(power_spectrum, cmap='gray')
for blob in inner_blobs:
    y, x, r = blob
    c = plt.Circle((x, y), r, color='red', linewidth=2, fill=False)
    plt.gca().add_patch(c)

for blob in outer_blobs:
    y, x, r = blob
    c = plt.Circle((x, y), r, color='blue', linewidth=2, fill=False)
    plt.gca().add_patch(c)

In [ ]:
# get angles for all the blobs, for grouping them
angles = [np.arctan2(blob[0] - center[0], blob[1] - center[1]) for blob in all_blobs]
angles = np.array(angles) * 180 / np.pi + 180


In [ ]:
# get the remainder after dividing by 60
angles = angles % 60

In [ ]:
plt.figure()
plt.hist(angles, bins=10)

In [ ]:
spots_A = [all_blobs[i] for i in range(len(all_blobs)) if angles[i] < 25]
spots_B = [all_blobs[i] for i in range(len(all_blobs)) if angles[i] > 25]

spots_A = np.array(spots_A)
spots_B = np.array(spots_B)

distances_A = [np.linalg.norm(np.array([blob[0], blob[1]]) - center) for blob in spots_A]
distances_B = [np.linalg.norm(np.array([blob[0], blob[1]]) - center) for blob in spots_B]

# this is the bit to make it checkerboard
outer_A = [spots_A[i] for i in range(len(spots_A)) if distances_A[i] > 30]
outer_B = [spots_B[i] for i in range(len(spots_B)) if distances_B[i] > 30]

inner_A = [spots_B[i] for i in range(len(spots_B)) if distances_B[i] < 30]
inner_B = [spots_A[i] for i in range(len(spots_A)) if distances_A[i] < 30]


spots_A = np.array(inner_A + outer_A)
spots_B = np.array(inner_B + outer_B)


spots_A = spots_A[:, 0:2]
spots_B = spots_B[:, 0:2]


In [ ]:
plt.figure()
plt.imshow(power_spectrum, cmap='gray')
plt.scatter([spot[1] for spot in spots_A], [spot[0] for spot in spots_A], color='red')
plt.scatter([spot[1] for spot in spots_B], [spot[0] for spot in spots_B], color='blue')

In [ ]:
reciprocal_scale = np.array([ft.get_slope(power_spectrum.u.values), ft.get_slope(power_spectrum.v.values)])
spots_A = spots_A * reciprocal_scale + [power_spectrum.u.values[0], power_spectrum.v.values[0]]
spots_B = spots_B * reciprocal_scale + [power_spectrum.u.values[0], power_spectrum.v.values[0]]

In [ ]:
filtered_A = pyTEMlib.image_tools.adaptive_fourier_filter(image, spots=spots_A, low_pass=1, reflection_radius=1)
view = filtered_A.plot(cmap='gray')

In [ ]:
filtered_power_spectrum = pyTEMlib.image_tools.power_spectrum(filtered_A, smoothing=0)
view = filtered_power_spectrum.plot()

In [ ]:
filtered_B = pyTEMlib.image_tools.adaptive_fourier_filter(image, spots=spots_B, low_pass=1, reflection_radius=1)
view = filtered_B.plot(cmap='gray')

In [ ]:
# in each of these, get the atoms and bond distances
# get the atoms
atoms_A = blob_log(filtered_A, min_sigma=4, max_sigma=10, num_sigma=10, threshold=0.9)
atoms_A = np.array(atoms_A[:, 0:2])

atoms_B = blob_log(filtered_B, min_sigma=4, max_sigma=10, num_sigma=10, threshold=0.9)
atoms_B = np.array(atoms_B[:, 0:2])

fig, ax = plt.subplots(1,2, sharex=True, sharey=True, figsize = (15,7))   
ax[0].imshow(image, cmap='gray')
ax[1].imshow(image, cmap='gray')
ax[0].scatter([atom[1] for atom in atoms_A], [atom[0] for atom in atoms_A], color='red',s=10)
ax[1].scatter([atom[1] for atom in atoms_B], [atom[0] for atom in atoms_B], color='blue',s=10)

for a in ax:
    a.axis('off')
fig.tight_layout()



In [ ]:
# import KDTree
from scipy.spatial import KDTree

In [ ]:
# Calculate the distance between the centroids
dist_A = []
tree = KDTree(atoms_A)
distances, indices = tree.query(atoms_A, k=3)
nearest_distances = distances[:, 1:] * float(pixel_size) * 1e10 # angstroms
dist_A.append(nearest_distances.flatten())
dist_A = np.array(dist_A).flatten()

dist_B = []
tree = KDTree(atoms_B)
distances, indices = tree.query(atoms_B, k=3)
nearest_distances = distances[:, 1:] * float(pixel_size) * 1e10 # angstroms
dist_B.append(nearest_distances.flatten())
dist_B = np.array(dist_B).flatten()

dist_total = np.concatenate((dist_A, dist_B))


In [ ]:
print('Median bond distance: ', np.median(dist_total))

In [ ]:
plt.figure()
plt.hist(dist_total, bins=40, color='gray', alpha=0.5)
plt.xlim(0.8,2)


## WSSe Exp data 

In [ ]:

# path = '/Users/austin/Desktop/Projects/WS2_twist/WS2_1deg/20230315/WS2_1deg/20230315 1045 STEM HAADF 9.20 Mx.emd'
path = '/Users/austin/Desktop/Projects/WS2_twist/WS2_15deg/20230303/20230303 1703 STEM HAADF 4.60 Mx.emd'
dset = ft.open_file(path)
image = dset['Channel_000']

# normalize the image
image = image - image.min()
image = image/image.max()
im_array = np.array(image)


view = image.plot()

pixel_size = dset['Channel_000'].original_metadata['BinaryResult']['PixelSize']['width'] # m/pixel
pixel_size = float(pixel_size) * 1e10 # Angstrom/pixel

print('Pixel size: ', pixel_size, ' Angstroms/pixel')

In [ ]:
# scipy ndimage zoom to get the right pixel size
from scipy.ndimage import zoom

zoom_factor = 2
zoomed_im = zoom(im_array, zoom_factor, order=3)
pixel_size = pixel_size / zoom_factor

In [ ]:
pixel_size

In [ ]:
plt.figure()
plt.imshow(zoomed_im, cmap='gray')

In [ ]:
rigid_registered_dataset = it.rigid_registration(image)
view = rigid_registered_dataset.plot()
    

In [ ]:
im_array = np.array(rigid_registered_dataset).sum(axis=0)
im_array.shape

In [ ]:
# im_array = np.array(rigid_registered_dataset).sum(axis=0)

# np.savez('WSSe_haadf_2.npz', im_array=zoomed_im, pixel_size=pixel_size)

In [ ]:
drift = rigid_registered_dataset.metadata['drift']
polynom_degree = 2 # 1 is linear fit, 2 is parabolic fit, ...

x = np.linspace(0,drift.shape[0]-1,drift.shape[0])

line_fit_x = np.polyfit(x, drift[:,0], polynom_degree)
poly_x = np.poly1d(line_fit_x)
line_fit_y = np.polyfit(x, drift[:,1], polynom_degree)
poly_y = np.poly1d(line_fit_y)

plt.figure()
plt.axhline(color = 'gray')
plt.plot(x, drift[:,0], label = 'drift x')
plt.plot(x, drift[:,1], label = 'drift y')
plt.plot(x, poly_x(x),  label = 'fit_drift_x')
plt.plot(x, poly_y(x),  label = 'fit_drift_y')

plt.legend();
ax_pixels = plt.gca()
ax_pixels.step(1, 1)

scaleX = (rigid_registered_dataset.x[1]-rigid_registered_dataset.x[0])*1000.  #in pm

ax_pm = ax_pixels.twinx()
x_1, x_2 = ax_pixels.get_ylim()

ax_pm.set_ylim(x_1*scaleX, x_2*scaleX)

ax_pixels.set_ylabel('drift [pixels]')
ax_pm.set_ylabel('drift [pm]')
ax_pixels.set_xlabel('image number');
plt.tight_layout()

In [ ]:
# non_rigid_registered = it.demon_registration(rigid_registered_dataset)

## get some more images of WSSe


In [ ]:
# WSSe Exp data 
path = '/Users/austin/Desktop/Projects/WS2_twist/WS2_1deg/20230315/WS2_1deg/20230315 1045 STEM HAADF 9.20 Mx.emd'
dset = ft.open_file(path)
image = dset['Channel_000']

# normalize the image
image = image - image.min()
image = image/image.max()
im_array = np.array(image)

view = image.plot()

pixel_size = dset['Channel_000'].original_metadata['BinaryResult']['PixelSize']['width'] # m/pixel
pixel_size = float(pixel_size) * 1e10 # Angstrom/pixel

print('Pixel size: ', pixel_size, ' Angstroms/pixel')



In [ ]:
rigid_registered_dataset = it.rigid_registration(image)

sum_im = rigid_registered_dataset.sum(axis=0)

# np.savez('WSSe_haadf.npz', im_array=im_array, pixel_size=pixel_size)

view = rigid_registered_dataset.plot()

In [ ]:
sum_im.data_type = 'image'

In [ ]:
power_spectrum = pyTEMlib.image_tools.power_spectrum(sum_im, smoothing=1)

power_spectrum.view_metadata()
view = power_spectrum.plot()

In [ ]:
from skimage.feature import blob_log
im_array = np.array(sum_im)

# find blobs
blobs = blob_log(power_spectrum, min_sigma=1, max_sigma=10, num_sigma=10, threshold=0.5)

# plot blobs
fig, ax = plt.subplots()
ax.imshow(power_spectrum, cmap='gray')
for blob in blobs:
    y, x, r = blob
    c = plt.Circle((x, y), r, color='red', linewidth=2, fill=False)
    ax.add_patch(c)

ax.set_xlim(power_spectrum.shape[1]//3, 2*power_spectrum.shape[1]//3)
ax.set_ylim(power_spectrum.shape[0]//3, 2*power_spectrum.shape[0]//3)
ax.axis('off')

In [ ]:
# get inner blobs:
inner_blobs = blob_log(power_spectrum, min_sigma=1, max_sigma=10, num_sigma=10, threshold=0.5)
# filter by distance from center
# center is blob closest to COM of the blobs
center = np.array([power_spectrum.shape[0]//2, power_spectrum.shape[1]//2])
distances = [np.linalg.norm(np.array([blob[0], blob[1]]) - center) for blob in inner_blobs]
inner_blobs = [inner_blobs[i] for i in range(len(inner_blobs)) if distances[i] < 40]
inner_blobs = [inner_blobs[i] for i in range(len(inner_blobs)) if distances[i] > 0]

# get outer blobs
outer_blobs = blob_log(power_spectrum, min_sigma=1, max_sigma=10, num_sigma=10, threshold=0.5)
# filter by distance from center
# center is blob closest to COM of the blobs
center = np.array([power_spectrum.shape[0]//2, power_spectrum.shape[1]//2])
distances = [np.linalg.norm(np.array([blob[0], blob[1]]) - center) for blob in outer_blobs]
outer_blobs = [outer_blobs[i] for i in range(len(outer_blobs)) if distances[i] > 40 and distances[i] < 80]





In [ ]:
all_blobs = inner_blobs + outer_blobs

In [ ]:
plt.figure()
plt.imshow(power_spectrum, cmap='gray')
for blob in inner_blobs:
    y, x, r = blob
    c = plt.Circle((x, y), r, color='red', linewidth=2, fill=False)
    plt.gca().add_patch(c)


for blob in outer_blobs:
    y, x, r = blob
    c = plt.Circle((x, y), r, color='blue', linewidth=2, fill=False)
    plt.gca().add_patch(c)

In [ ]:
# Find the nearest neighbor for each blob
tree = KDTree(all_blobs)
distances, indices = tree.query(all_blobs, k=2)  # k=2 to include the point itself as the closest
pairs = {(min(i, indices[i][1]), max(i, indices[i][1])) for i in range(len(all_blobs))}

plt.figure(figsize=(8, 8))
colors = cm.rainbow(np.linspace(0, 1, len(pairs)))
for idx, pair in enumerate(pairs):
    plt.scatter([all_blobs[pair[0]][1]], [all_blobs[pair[0]][0]], color=colors[idx])
    plt.scatter([all_blobs[pair[1]][1]], [all_blobs[pair[1]][0]], color=colors[idx])
plt.scatter(center[1], center[0], color='black', marker='x')



In [ ]:
center

In [ ]:
smaller_angle_coords = []
larger_angle_coords = []

for pair in pairs:
    angle1 = np.arctan2(all_blobs[pair[0]][0] - center[0], all_blobs[pair[0]][1] - center[1]) * 180 / np.pi
    angle2 = np.arctan2(all_blobs[pair[1]][0] - center[0], all_blobs[pair[1]][1] - center[1]) * 180 / np.pi

    if angle1 < 0:
        angle1 += 360
    if angle2 < 0:
        angle2 += 360

    # Compare angles to determine which is smaller in the circular sense
    angle_diff = (angle2 - angle1 + 360) % 360

    avg_dist_from_center = (np.linalg.norm(np.array(all_blobs[pair[0]])[:2] - center) + np.linalg.norm(np.array(all_blobs[pair[1]])[:2] - center)) / 2

    if avg_dist_from_center < 40:
        if angle_diff < 180:
            smaller_angle_coords.append(all_blobs[pair[0]])
            larger_angle_coords.append(all_blobs[pair[1]])
        else:
            smaller_angle_coords.append(all_blobs[pair[1]])
            larger_angle_coords.append(all_blobs[pair[0]])

    else:
        if angle_diff > 180:
            smaller_angle_coords.append(all_blobs[pair[0]])
            larger_angle_coords.append(all_blobs[pair[1]])
        else:
            smaller_angle_coords.append(all_blobs[pair[1]])
            larger_angle_coords.append(all_blobs[pair[0]])
        

In [ ]:
plt.figure()
plt.imshow(power_spectrum, cmap='gray')

for blob in smaller_angle_coords:
    y, x, r = blob
    c = plt.Circle((x, y), r, color='red', linewidth=2, fill=False)
    plt.gca().add_patch(c)

for blob in larger_angle_coords:
    y, x, r = blob
    c = plt.Circle((x, y), r, color='blue', linewidth=2, fill=False)
    plt.gca().add_patch(c)

In [ ]:
spots_A = np.array(smaller_angle_coords)[:,:2]
spots_B = np.array(larger_angle_coords)[:,:2]

In [ ]:
reciprocal_scale = np.array([ft.get_slope(power_spectrum.u.values), ft.get_slope(power_spectrum.v.values)])
spots_A = spots_A * reciprocal_scale + [power_spectrum.u.values[0], power_spectrum.v.values[0]]
spots_B = spots_B * reciprocal_scale + [power_spectrum.u.values[0], power_spectrum.v.values[0]]



In [ ]:
filtered_A = pyTEMlib.image_tools.adaptive_fourier_filter(sum_im, spots=spots_A, low_pass=1, reflection_radius=1)
view = filtered_A.plot(cmap='gray')

In [ ]:
filtered_power_spectrum = pyTEMlib.image_tools.power_spectrum(filtered_A, smoothing=0)
view = filtered_power_spectrum.plot()

In [ ]:
filtered_B = pyTEMlib.image_tools.adaptive_fourier_filter(sum_im, spots=spots_B, low_pass=1, reflection_radius=1)
view = filtered_B.plot(cmap='gray')

In [ ]:
filtered_power_spectrum = pyTEMlib.image_tools.power_spectrum(filtered_B, smoothing=0)
view = filtered_power_spectrum.plot()

In [ ]:
# in each of these, get the atoms and bond distances
# get the atoms
atoms_A = blob_log(filtered_A, min_sigma=4, max_sigma=10, num_sigma=10, threshold=0.01)
atoms_A = np.array(atoms_A[:, 0:2])

atoms_B = blob_log(filtered_B, min_sigma=4, max_sigma=10, num_sigma=10, threshold=0.08)
atoms_B = np.array(atoms_B[:, 0:2])

fig, ax = plt.subplots(1,2, sharex=True, sharey=True, figsize = (15,7))   
ax[0].imshow(filtered_A, cmap='gray')
ax[1].imshow(filtered_B, cmap='gray')
ax[0].scatter([atom[1] for atom in atoms_A], [atom[0] for atom in atoms_A], color='red',s=10)
ax[1].scatter([atom[1] for atom in atoms_B], [atom[0] for atom in atoms_B], color='blue',s=10)

for a in ax:
    a.axis('off')
fig.tight_layout()



In [ ]:
from skimage.draw import disk

In [ ]:
im = np.array(sum_im)
# Radius of the circle
r = 3

# Create an empty list to store the summed intensities
intensities = []

# Set up plot for visualizing
fig, ax = plt.subplots()
ax.imshow(im, cmap='gray')

# Iterate through each point
for point in atoms_A:
    row, col = point
    # Create a circular mask centered at the point
    rr, cc = disk((row, col), r, shape=im.shape)
    
    # Sum the intensity within the circle
    intensity = np.sum(im[rr, cc])
    intensities.append(intensity)
    
    # Visualize the circle on the image
    circle = plt.Circle((col, row), r, color='red', fill=False)
    ax.add_patch(circle)

plt.figure()
plt.hist(intensities, bins=20, alpha=0.5)


In [ ]:
im = np.array(sum_im)
# Radius of the circle
r = 3

# Create an empty list to store the summed intensities
intensities = []

# Set up plot for visualizing
fig, ax = plt.subplots()
ax.imshow(im, cmap='gray')

# Iterate through each point
for point in atoms_B:
    row, col = point
    # Create a circular mask centered at the point
    rr, cc = disk((row, col), r, shape=im.shape)
    
    # Sum the intensity within the circle
    intensity = np.sum(im[rr, cc])
    intensities.append(intensity)
    
    # Visualize the circle on the image
    circle = plt.Circle((col, row), r, color='b', fill=False)
    ax.add_patch(circle)

plt.figure()
plt.hist(intensities, bins=20, alpha=0.5)

In [ ]:
im = np.array(filtered_A)
# Radius of the circle
r = 3

# Create an empty list to store the summed intensities
intensities = []

# Set up plot for visualizing
fig, ax = plt.subplots()
ax.imshow(im, cmap='gray')

# Iterate through each point
for point in atoms_A:
    row, col = point
    # Create a circular mask centered at the point
    rr, cc = disk((row, col), r, shape=im.shape)
    
    # Sum the intensity within the circle
    intensity = np.sum(im[rr, cc])
    intensities.append(intensity)
    
    # Visualize the circle on the image
    circle = plt.Circle((col, row), r, color='r', fill=False)
    ax.add_patch(circle)

plt.figure()
plt.hist(intensities, bins=50, alpha=0.5)

In [ ]:
im = np.array(filtered_B)
# Radius of the circle
r = 3

# Create an empty list to store the summed intensities
intensities = []

# Set up plot for visualizing
fig, ax = plt.subplots()
ax.imshow(im, cmap='gray')

# Iterate through each point
for point in atoms_B:
    row, col = point
    # Create a circular mask centered at the point
    rr, cc = disk((row, col), r, shape=im.shape)
    
    # Sum the intensity within the circle
    intensity = np.sum(im[rr, cc])
    intensities.append(intensity)
    
    # Visualize the circle on the image
    circle = plt.Circle((col, row), r, color='r', fill=False)
    ax.add_patch(circle)

plt.figure()
plt.hist(intensities, bins=50, alpha=0.5)